In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/depression-dataset-processed/df_clean.csv


In [2]:
import pandas as pd

df_clean = pd.read_csv('/kaggle/input/depression-dataset-processed/df_clean.csv')
df_clean.shape

(128930, 353)

In [3]:
from imblearn.over_sampling import ADASYN

X = df_clean.drop(columns=['Depression'])
y = df_clean.loc[:, 'Depression']
print(X.shape, y.shape)
adasyn = ADASYN(sampling_strategy='minority', n_jobs=-1, n_neighbors=5, random_state=32)
X_sampled, y_sampled = adasyn.fit_resample(X, y)
X_sampled.shape, y_sampled.shape

(128930, 352) (128930,)


/opt/conda/lib/python3.10/site-packages/imblearn/over_sampling/_adasyn.py:156: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(


((214264, 352), (214264,))

In [4]:
df_sampled = pd.DataFrame(X_sampled, columns=[col for col in df_clean.columns if col != "Depression"])
df_sampled["Depression"] = y_sampled

df_sampled.shape

/tmp/ipykernel_30/991659502.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_sampled["Depression"] = y_sampled


(214264, 353)

In [5]:
# df_sampled.Depression.value_counts()
(df_sampled.isna().sum() > 0 ).sum()

0

In [6]:
corr = X_sampled.corrwith(y_sampled)

/opt/conda/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/conda/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [7]:
corr = abs(corr)
corr = corr.fillna(0)
corr = corr.sort_values(ascending=False)

In [8]:
import matplotlib.pyplot as plt

u_corr = corr[corr >= 0.08]
# plt.scatter(range(len(u_corr)), u_corr)
# plt.show()
useful_cols = u_corr.index.tolist()
useful_cols

['Age',
 'Have you ever had suicidal thoughts ?_Yes',
 'Working Professional or Student_Student',
 'Working Professional or Student_Working Professional',
 'Profession_Teacher',
 'Have you ever had suicidal thoughts ?_No',
 'Job Satisfaction',
 'Degree_Class 12',
 'Dietary Habits_Unhealthy',
 'Sleep Duration_Less than 5 hours',
 'Study Satisfaction',
 'Work/Study Hours',
 'Financial Stress',
 'Profession_Content Writer',
 'City_Hyderabad',
 'Family History of Mental Illness_Yes',
 'Gender_Male',
 'City_Ahmedabad',
 'City_Srinagar',
 'Academic Pressure',
 'City_Thane',
 'Sleep Duration_7-8 hours',
 'City_Bhopal']

In [9]:
X_useful = df_sampled[useful_cols]
y_useful = df_sampled['Depression']
X_useful.shape, y_useful.shape

((214264, 23), (214264,))

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

pipe = Pipeline([
    ('poly', PolynomialFeatures(include_bias=False, interaction_only=False, degree=2)),
     ('scaler', StandardScaler())
])
X_scaled = pipe.fit_transform(X_useful)
y_scaled = y_useful

In [11]:
X_scaled.shape, y_scaled.shape

((214264, 299), (214264,))

In [12]:
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=1, random_state=32, test_size=0.3)
for tr, te in sss.split(X_scaled, y_scaled):
    X_full, y_full = X_scaled[tr], y_scaled.iloc[tr]
    X_test, y_test = X_scaled[te], y_scaled.iloc[te]
for tr, val in sss.split(X_full, y_full):
    X_train, y_train = X_full[tr], y_full.iloc[tr]
    X_val, y_val = X_full[val], y_full.iloc[val]
# X_train[X_train.select_dtypes(exclude=['float']).columns] = X_train[X_train.select_dtypes(exclude=['float']).columns].astype(int)
# X_test[X_test.select_dtypes(exclude=['float']).columns] = X_test[X_test.select_dtypes(exclude=['float']).columns].astype(int)

print(X_train.shape, X_test.shape, X_val.shape)

(104988, 299) (64280, 299) (44996, 299)


In [13]:
import keras
from keras import layers, activations, initializers, regularizers
import tensorflow as tf
import numpy as np

def build_model(input_shape, n_layers, units, n_classes, drop_rate):
    input_layer = layers.Input(shape=input_shape)

    x = input_layer
    for _ in range(n_layers):
        if units < 25:
            units = 25
        x = layers.Dense(
            units,
            activation=activations.leaky_relu,
            kernel_initializer=initializers.he_normal,
            kernel_regularizer=regularizers.l2(0.01)
        )(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(rate=drop_rate)(x)
        units //= 2
    
    if n_classes > 2:
        activation = activations.softmax
    else:
        activation = activations.sigmoid
    output_layer = layers.Dense(n_classes, activation=activation, kernel_initializer=initializers.GlorotNormal)(x)

    model = keras.Model(inputs=input_layer, outputs=output_layer)
    return model

In [14]:
input_shape  = (X_train.shape[1],)
input_shape

(299,)

In [15]:
model = build_model(input_shape=input_shape, n_layers=2, units=300, n_classes=1, drop_rate=0.3)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 299)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 300)            │        90,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 300)            │         1,200 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 300)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 150)            │        45,150 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 150)            │           600 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 150)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           151 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 137,101 (535.55 KB)

 Trainable params: 136,201 (532.04 KB)

 Non-trainable params: 900 (3.52 KB)

In [22]:
import optuna
import keras
from keras import (
    layers,
    activations,
    initializers,
    regularizers,
    optimizers,
    losses,
    metrics,
)
import tensorflow as tf
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit


def objective(trial):
    strategy = tf.distribute.MirroredStrategy()
    with strategy.scope():
        n_layers = trial.suggest_int("n_layers", 2, 10)
        units = trial.suggest_int("units", 100, 1000)
        drop_out_rate = trial.suggest_float("drop_out_rate", 0.0, 0.6)
        learning_rate = trial.suggest_float("learning_rate", 1e-7, 1e-2, log=True)
        
        epochs = trial.suggest_int("epochs", 10, 1000)
        shuffle_size = 5000
        batch_size = 512 * 2
        train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
        val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
        train = (
            train_dataset.shuffle(shuffle_size)
            .batch(batch_size)
            .prefetch(tf.data.experimental.AUTOTUNE)
        )
        val = (
            val_dataset.shuffle(shuffle_size)
            .batch(batch_size)
            .prefetch(tf.data.experimental.AUTOTUNE)
        )

        model = build_model(
            input_shape=input_shape,
            n_layers=n_layers,
            units=units,
            n_classes=1,
            drop_rate=drop_out_rate,
        )

        opt = optimizers.Adam(learning_rate=learning_rate)
        loss = losses.BinaryCrossentropy
        metric = [metrics.binary_accuracy]
        model.compile(optimizer=opt, loss=loss, metrics=metric)

        history = model.fit(train, epochs=epochs, validation_data=val, verbose=2)
        
        val_loss = history.history['val_loss']
        train_loss = history.history['loss']
    return train_loss, val_loss

sampler = optuna.samplers.TPESampler(n_startup_trials=50)
study = optuna.create_study(sampler=sampler,directions=['minimize', 'minimize'], study_name='depression_study_ann', storage='sqlite:///depression_study_ann.db', load_if_exists=True)
study.optimize(objective, n_trials=500 )


[I 2024-11-14 10:08:12,663] Using an existing study with name 'depression_study_ann' instead of creating a new one.


Epoch 1/45
103/103 ━━━━━━━━━━━━━━━━━━━━ 30s 57ms/step - binary_accuracy: 0.5022 - loss: 18.7878 - val_binary_accuracy: 0.5110 - val_loss: 18.3751
Epoch 2/45
103/103 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - binary_accuracy: 0.5018 - loss: 18.7781 - val_binary_accuracy: 0.5019 - val_loss: 18.3748
Epoch 3/45
103/103 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - binary_accuracy: 0.5005 - loss: 18.7833 - val_binary_accuracy: 0.5036 - val_loss: 18.3723
Epoch 4/45
103/103 ━━━━━━━━━━━━━━━━━━━━ 6s 52ms/step - binary_accuracy: 0.5031 - loss: 18.7746 - val_binary_accuracy: 0.5008 - val_loss: 18.3713
Epoch 5/45
103/103 ━━━━━━━━━━━━━━━━━━━━ 5s 47ms/step - binary_accuracy: 0.5024 - loss: 18.7688 - val_binary_accuracy: 0.5032 - val_loss: 18.3690
Epoch 6/45
103/103 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - binary_accuracy: 0.5027 - loss: 18.7675 - val_binary_accuracy: 0.5032 - val_loss: 18.3670
Epoch 7/45
103/103 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - binary_accuracy: 0.4991 - loss: 18.7803 - val_binary_accuracy: 0.5033 - va

[W 2024-11-14 10:09:25,259] Trial 4 failed with parameters: {'n_layers': 10, 'units': 393, 'drop_out_rate': 0.5925318000720806, 'learning_rate': 1.8124293111467187e-07, 'epochs': 45} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_30/2514753221.py", line 39, in objective
    history = model.fit(train, epochs=epochs, validation_data=val)
  File "/opt/conda/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 117, in error_handler
    return fn(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/keras/src/backend/tensorflow/trainer.py", line 314, in fit
    logs = self.train_function(iterator)
  File "/opt/conda/lib/python3.10/site-packages/tensorflow/python/util/traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)
  File "/opt/conda/lib/

KeyboardInterrupt: 

In [ ]:
test = test_dataset.shuffle(1000).batch(128).prefetch(tf.data.experimental.AUTOTUNE)

model.evaluate(test)